In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
# import lightgbm as lgb
# import xgboost as xgb
import scipy
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
# import statsmodels.formula.api as smf
import matplotlib as mpl
import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import seaborn as sns
# import dill
import datetime as dt
# import pickle
# import random
# import math
# import time
# import os
# import re
import colorsys
import math
import collections
# from statsmodels.nonparametric.smoothers_lowess import lowess

from sklearn.metrics import root_mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [3]:
def saturate_color(color_rgb, saturation=0.75):
    color_hls = colorsys.rgb_to_hls(
        color_rgb[0], color_rgb[1], color_rgb[2])
    color_hls_saturated = (
        color_hls[0], color_hls[1], saturation*color_hls[2])
    color_rgb_saturated = colorsys.hls_to_rgb(
        color_hls_saturated[0], color_hls_saturated[1], color_hls_saturated[2])
    return color_rgb_saturated


def saturate_palette(palette, saturation=0.75):
    palette_saturated = [saturate_color(i, saturation=saturation) for i in palette]
    return palette_saturated

In [4]:
def alpha_color(color, alpha=0.75):
    new_color = tuple (x + (1 - x) * (1 - alpha) for x in color)
    return new_color


def alpha_palette(palette, alpha=0.75):
    palette_alphed = [alpha_color(i, alpha=alpha) for i in palette]
    return palette_alphed

In [5]:
theme = 'light'
# theme = 'dark'

scatter = {
    'lw': 0.15
}
point = {
    'linestyle': 'none',
    'markersize': 3,
    'err_kws': {'lw': 1.2},
    'capsize': 0.03
}
hist = {
    'alpha': 1
}
title_inline = {
    'pad': 25
}
custom_params = {
    'figure.figsize': (10, 2.5), # ширина и высота в дюймах
    'figure.titlesize': 12,
    'figure.titleweight': 'bold',
    'figure.labelsize': 9,
    'figure.labelweight': 'bold',
    'axes.axisbelow': True,
    'axes.spines.bottom': True, # ось X
    'axes.spines.left': True, # ось Y
    'axes.spines.right': False, # не отображать правую рамку 
    'axes.spines.top': False, # не отображать верхнюю рамку
    'axes.linewidth': 1.2, # 1.2
    'axes.titlelocation': 'left', # расположение заголовка
    'axes.titlepad': 15.0, # расстояние между заголовком и графиком
    'axes.titlesize': 10.0, # заголовок
    'axes.titleweight': 'bold', # заголовок
    'axes.labelpad': 8.0, # расстояние между подписью оси и осью
    'axes.labelsize': 9.0, # оси
    'axes.labelweight': 'bold', # подписи осей
    'xaxis.labellocation': 'left', # положение подписи
    'yaxis.labellocation': 'top', # положение подписи
    # 'axes.xmargin': 0.025, # отстутпы по оси X
    # 'axes.ymargin': 0.085, # отстутпы по оси Y
    'axes.grid': True,
    'axes.grid.axis': 'both', # какую разметку отображать
    'grid.linestyle': ':',
    'grid.linewidth': 0.5,
    'lines.linewidth': 1,
    'lines.markersize': 3.5,
    'xtick.bottom': False,
    'xtick.direction': 'in',
    'xtick.major.bottom': True,
    'xtick.major.pad': 8,
    'xtick.major.size': 1.5,
    'xtick.major.top': True,
    'xtick.major.width': 0.75,
    'xtick.labelsize': 8.0, # деления оси X
    'ytick.left': False,
    'ytick.direction': 'in',
    'ytick.major.left': True,
    'ytick.major.pad': 8,
    'ytick.major.size': 1.5,
    'ytick.major.left': True,
    'ytick.major.width': 0.75,
    'ytick.labelsize': 8.0, # деления оси Y
    'legend.title_fontsize': 8.0,
    'legend.fontsize': 8.0,
    'legend.framealpha': 0.75,
    'font.family': 'Ubuntu',
    'scatter.marker': '.'
}
custom_params_light = {
    'figure.facecolor': '#FEFEFE', # FEFEFE
    'figure.edgecolor': '#FEFEFE',
    'axes.facecolor': '#FEFEFE',
    'axes.titlecolor': '#606060',
    'axes.labelcolor': '#505050',
    'axes.edgecolor': '#DDDDDD', # цвет осей
    'grid.color': '#D9D9D9',  
    'xtick.color': '#BBBBBB', # 0.90, BBBBBB
    'xtick.labelcolor': '#9F9F9F',
    'ytick.color': '#BBBBBB', # 0.90
    'ytick.labelcolor': '#9F9F9F',
    'legend.labelcolor': '#4B4B4B',
    'legend.edgecolor': '#A6A6A6',
    'legend.facecolor': '#FEFEFE',
    'text.color': '#4B4B4B',
    'patch.edgecolor': '#FEFEFE', # цвет границы стобцов, 
    'patch.force_edgecolor': True,
    'scatter.edgecolors': 'face'
}
custom_params_dark = {
    'figure.facecolor': '#383838',
    'axes.facecolor': '#383838',
    'axes.edgecolor': '#555555',
    'axes.titlecolor': '#AAAAAA',
    'axes.labelcolor': '#AAAAAA',
    'grid.color': '#CCCCCC', # CCCCCC
    'grid.linewidth': 0.1,
    'xtick.color': '#858585',
    'ytick.color': '#858585',
    'lines.markersize': 3.5,
    'legend.edgecolor': '#555555',
    'legend.facecolor': '#383838',
    'legend.labelcolor': '#858585',
    'legend.framealpha': 0.5,
    'text.color': '#AAAAAA',
    'patch.edgecolor': '#383838'
}
if theme == 'light':
    # 4E77AD
    palette_bright = sns.color_palette([
    #    0 grey     1 red      2 blue     3 green    4 yellow   
        '#4B4B4B', '#AF4035', '#4E77AD', '#00937F', '#FFA500',  
    #    5 purple   6 khaki    7 blue     8 pink     9 orange
        '#704A99', '#AF966D', '#124089', '#D07190', '#DE6535',
    #   -10 dark   -9 blue    -8 blue    -7 blue    -6 ligt
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
    #   -5 dark    -4 grey    -3 grey    -2 grey    -1 light
        '#303030', '#505050', '#707070', '#909090', '#BBBBBB'
    ])
    palette = saturate_palette(palette_bright, saturation=0.85)
    palette = alpha_palette(palette, alpha=0.90)
    # add palette to custom parameters dictionary for light theme
    custom_params_light.update({'axes.prop_cycle': plt.cycler(color=palette)})

    corr_matrix = sns.color_palette(
        'blend:#255986,#FFFFFF,#933737', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#FFFFFF,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of light theme to parameters dictionary
    custom_params.update(custom_params_light)
    # update rcParams 
    plt.rcParams.update(custom_params)

if theme == 'dark':
    
    palette = sns.color_palette([
    #    1 blue     2 red      3 green    4 yellow  5 seal
        '#7390B6', '#CB6F6F', '#5DA167', '#FFFF88', '#2BAFA2',
        '#9A6484', '#5B3256', '#6C4839', '#5C8DA0', '#847637',
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
        '#505050', '#606060', '#707070', '#808080', '#AAAAAA',
    ])
    palette = saturate_palette(palette, saturation=0.85)
    # add palette to custom parameters dictionary for dark theme
    custom_params_dark.update({'axes.prop_cycle': plt.cycler(color=palette)})
    
    corr_matrix = sns.color_palette(
        'blend:#285780,#909090,#8D3131', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#CCCCCC,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of dark theme to parameters dictionary
    custom_params.update(custom_params_dark)
    # update rcParams 
    plt.rcParams.update(custom_params)

%matplotlib inline

seed = 42

In [6]:
sns.color_palette(palette[:5])

[(0.36470588235294116, 0.36470588235294116, 0.36470588235294116),
 (0.6853529411764706, 0.3523529411764706, 0.3193529411764706),
 (0.40044117647058824, 0.5234411764705883, 0.6854411764705882),
 (0.13891176470588232, 0.5799117647058823, 0.5199117647058825),
 (0.9325, 0.6625, 0.16749999999999993)]

In [7]:
sns.color_palette(palette[5:10])

[(0.4960882352941175, 0.38208823529411756, 0.6190882352941176),
 (0.7001764705882353, 0.6251764705882352, 0.5021764705882353),
 (0.19502941176470587, 0.33302941176470585, 0.5520294117647058),
 (0.808970588235294, 0.5239705882352942, 0.6169705882352938),
 (0.8387941176470588, 0.47579411764705876, 0.3317941176470588)]

In [8]:
sns.color_palette(palette[10:15])

[(0.28423529411764703, 0.37423529411764694, 0.452235294117647),
 (0.3340882352941177, 0.44808823529411757, 0.5470882352941177),
 (0.39585294117647046, 0.5398529411764704, 0.6688529411764705),
 (0.507470588235294, 0.645470588235294, 0.7654705882352942),
 (0.7066176470588236, 0.8146176470588233, 0.9016176470588235)]

In [9]:
sns.color_palette(palette[15:])

[(0.26941176470588235, 0.26941176470588235, 0.26941176470588235),
 (0.38235294117647056, 0.38235294117647056, 0.38235294117647056),
 (0.49529411764705883, 0.49529411764705883, 0.49529411764705883),
 (0.608235294117647, 0.608235294117647, 0.608235294117647),
 (0.7599999999999999, 0.7599999999999999, 0.7599999999999999)]

### Additional elements

In [10]:
handle_april = Line2D(
    [], [], label='April', lw=1, color=palette[0], alpha=0.75)
handle_may = Line2D(
    [], [], label='May', lw=1, color=palette[1], alpha=0.75)
handle_june = Line2D(
    [], [], label='June', lw=1, color=palette[2], alpha=0.75)
handle_july = Line2D(
    [], [], label='July', lw=1, color=palette[3], alpha=0.75)

handles_4_months = [handle_april, handle_may, handle_june, handle_july]

In [14]:
weekday_names_dict = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday',
}